In [1]:
import json
import requests
import pandas as pd
import pprint
import pyodbc
import datetime

In [2]:
shopping = pd.read_csv('shopping_mall_address.csv')
shopping.head()

,address
0,100 AM
1,313@Somerset
2,Aperia
3,Balestier Hill Shopping Centre
4,Bugis Cube


In [3]:
text_replace_dict = {'GEORGES':"GEORGE'S",
                     'CWEALTH':'COMMONWEALTH',
                     'QUEENS':"QUEEN'S",
                     'MAMOR':"MA'MOR",
                     ' ST ':' STREET '
                    }

def clean_address(string,dictionary):
     for item in string.split(' '):
            
# print(item)
        if item in text_replace_dict.keys():
            string = string.replace(item,text_replace_dict[item])
            
        else:
            string
            
        return string

shopping['address_clean'] = [clean_address(val,text_replace_dict) for val in shopping['address']]


In [4]:
shopping = shopping.drop_duplicates('address')

In [5]:
for index,add in enumerate(set(shopping['address_clean'])):
    url= 'https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal='+add
    print(index,url)
    response = requests.get(url)
    data = json.loads(response.text) 
 
    temp_df = pd.DataFrame.from_dict(data['results'])
    temp_df['address'] = add
 
    if index == 0:
        address_geoloc = temp_df
    
    else:
        address_geoloc = address_geoloc.append(temp_df)
        address_geoloc.to_csv('shopping_address_geoloc.csv')

0 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Katong Square
1 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Suntec City
2 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=City Square Mall
3 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Loyang Point
4 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Woodlands Civic Centre
5 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Sun Plaza
6 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=ION Orchard
7 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=White Sands
8 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Mille

74 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Jem
75 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Katong V
76 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=100 AM
77 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Knightsbridge[1]
78 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=The Shoppes at Marina Bay Sands
79 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=313@Somerset
80 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=321 Clementi
81 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=OD Mall
82 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Tiong 

147 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=The Midtown
148 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Leisure Park Kallang
149 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Admiralty Place
150 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Causeway Point
151 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Waterway Point
152 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Jurong Point
153 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=The South Beach
154 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=JCube
155 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNu

In [7]:
geo = pd.read_csv('shopping_address_geoloc.csv')
shopping_2 = shopping.merge(geo[geo['ADD_SEQ']==0],how='left',left_on='address',right_on='address')

In [8]:
shopping_2.to_csv('shopping_geoloc.csv')